<a href="https://colab.research.google.com/github/simulate111/Introduction-to-Human-Language-Technology/blob/main/course_project_Reza2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to HLT Project (Template)

- Student(s) Name(s): Mohammadreza Akhtari
- Date: June 2024
- Chosen Corpus: imdb
- Contributions (if group project): -

### Corpus information

- Description of the chosen corpus: Large Movie Review Dataset. This is a dataset for binary sentiment classification. A set of 25,000 highly polar movie reviews for training, and 25,000 for testing.
- Paper(s) and other published materials related to the corpus: Maas, Andrew, et al. "Learning word vectors for sentiment analysis." Proceedings of the 49th annual meeting of the association for computational linguistics: Human language technologies. 2011.
- State-of-the-art performance (best published results) on this corpus: Achieving an f1-score of 77.66% and an accuracy of 77.81% to predict positive or negaitve sentiments according to the recent publicaiton in 2023 of "Opinion classification for IMDb review based using naive bayes method" (https://doi.org/10.1063/5.0171628).

---

## 1. Setup

In [1]:
# Your code to install and import libraries etc. here
!pip3 install -q transformers[torch] datasets evaluate optuna plotly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 50.3 MB/s eta 0:00:00
ERROR: pip's depen

---

## 2. Data download and preprocessing

### 2.1. Download the corpus

In [2]:
# Your code to download the corpus here
from pprint import pprint #pprint
import datasets
dset1=datasets.load_dataset("imdb")
pprint(dset1)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [3]:
dset=dset1.shuffle() #This is never a bad idea, datasets may have ordering to them, which is not what we want
del dset["unsupervised"] #Delete the unlabeled part of the dataset, we don't need it for anything

In [4]:
pprint(dset['train'][0]['text'])
print(dset['train'][0]['label'])

('After looking at monkeys (oops apes) for more than one hour, I was feeling '
 'like one too. I was an ape, spending money on this movie. Please people, '
 'hold you money in your pocket and go see some funny movie like Bridget '
 "Jones's Diary..")
0


### 2.2. Preprocessing

In [5]:
#Tokenize and map vocabulary
import sklearn.feature_extraction

vectorizer=sklearn.feature_extraction.text.CountVectorizer(binary=True,max_features=20000)

texts=[ex["text"] for ex in dset["train"]] #get a list of all texts from the training data
vectorizer.fit(texts) #"Trains" the vectorizer, i.e. builds its vocabulary

CountVectorizer(binary=True, max_features=20000)

In [6]:
#Building the feature vectors
def vectorize_example(ex):
    vectorized=vectorizer.transform([ex["text"]]) # [...] because the vectorizer expects a list/iterable over inputs, not one input
    non_zero_features=vectorized.nonzero()[1] #.nonzero gives a pair of (rows,columns), we want the columns
    non_zero_features+=1 #feature index 0 will have a special meaning
                         # so let us not produce it by adding +1 to everything
    return {"input_ids":non_zero_features}

vectorized=vectorize_example(dset["train"][0])

In [7]:
print(vectorized)

{'input_ids': array([  598,   863,   891,  1020,  1021,  1302,  2391,  5075,  6772,
        7139,  7403,  7743,  8625,  8758,  9084,  9795, 10453, 10624,
       11592, 11599, 11657, 11746, 12421, 12423, 12440, 13023, 13336,
       13374, 15678, 16533, 16701, 17879, 17961, 18163, 19396, 19926,
       19932], dtype=int32)}


In [8]:
# We can map back to vocabulary and check that everything works
# vectorizer.vocabulary_ is a dictionary {key:word, value:idx}

idx2word=dict((i,w) for (w,i) in vectorizer.vocabulary_.items()) #inverse the vocab dictionary
words=[]
for idx in vectorized["input_ids"]:
    words.append(idx2word[idx-1]) ## It is easy to forgot we moved all by +1
pprint(", ".join(words)) #This is now the bag of words representation of the document

('after, an, and, ape, apes, at, bridget, diary, feeling, for, funny, go, '
 'hold, hour, in, jones, like, looking, money, monkeys, more, movie, on, one, '
 'oops, people, please, pocket, see, some, spending, than, this, too, was, '
 'you, your')


In [9]:
# Apply the tokenizer to the whole dataset using .map()
dset_tokenized = dset.map(vectorize_example,num_proc=4)
pprint(dset_tokenized["train"][0])

Map (num_proc=4):   0%|          | 0/25000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/25000 [00:00<?, ? examples/s]

{'input_ids': [598,
               863,
               891,
               1020,
               1021,
               1302,
               2391,
               5075,
               6772,
               7139,
               7403,
               7743,
               8625,
               8758,
               9084,
               9795,
               10453,
               10624,
               11592,
               11599,
               11657,
               11746,
               12421,
               12423,
               12440,
               13023,
               13336,
               13374,
               15678,
               16533,
               16701,
               17879,
               17961,
               18163,
               19396,
               19926,
               19932],
 'label': 0,
 'text': 'After looking at monkeys (oops apes) for more than one hour, I was '
         'feeling like one too. I was an ape, spending money on this movie. '
         'Please people, hold you 

In [10]:
#Padding and Collation (forming a batch)
import torch

def collator(list_of_examples):
    batch={"labels":torch.tensor(list(ex["label"] for ex in list_of_examples))} #this is easy, labels are made into a single tensor
    #the worse bit is now to pad the examples, as they are of different length
    tensors=[]
    max_len=max(len(example["input_ids"]) for example in list_of_examples) #this is the longest example in the batch
    #everything needs to be padded to fit in length the longest example
    #(so we can build a single tensor out of it)
    for example in list_of_examples:
        ids=torch.tensor(example["input_ids"]) #pick the input ids
        # pad(what,(from_left, from_right)) <- this is how we call the stock pad function
        padded=torch.nn.functional.pad(ids,(0,max_len-ids.shape[0])) #pad by max - current length, pads with zero by default
        tensors.append(padded) #accumulated the padded ids
    batch["input_ids"]=torch.vstack(tensors) #now that we have all of them the same length, a simple vstack() stacks them up
    return batch #...and that's all there is to it

In [11]:
#Build a batch from 2 examples, with padding
batch=collator([dset_tokenized["train"][2],dset_tokenized["train"][7]])
print("Shape of labels:",batch["labels"].shape)
print("Shape of input_ids:",batch["input_ids"].shape)
pprint(batch["labels"])
pprint(batch["input_ids"])

Shape of labels: torch.Size([2])
Shape of input_ids: torch.Size([2, 247])
tensor([1, 1])
tensor([[  136,   420,   437,   891,   963,  1004,  1117,  1156,  1209,  1302,
          1864,  2612,  2633,  3051,  3054,  3066,  3076,  3432,  3457,  3898,
          4936,  4993,  5436,  5460,  5461,  5753,  6051,  6274,  6577,  6595,
          6697,  6901,  6947,  7139,  7166,  7608,  7825,  7910,  8217,  8311,
          8639,  9056,  9084,  9595,  9621, 10005, 10219, 10306, 10453, 10958,
         11069, 11164, 11266, 11657, 11701, 11746, 11749, 11799, 11955, 12202,
         12300, 12348, 12423, 12446, 12492, 12856, 13023, 13095, 13277, 14339,
         14554, 14710, 15575, 15678, 16051, 16469, 16512, 17620, 17887, 17891,
         17901, 17938, 17961, 17973, 18066, 18073, 18112, 18164, 18971, 19318,
         19396, 19444, 19538, 19550, 19593, 19711, 19761, 19863, 19926,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0

---

## 3. Machine learning model

### 3.1. Model training

In [12]:
#Build the MLP model
import torch
import transformers

# A model wants a config, I can simply inherit from the base
# class for pretrained configs
class MLPConfig(transformers.PretrainedConfig):
    pass

# This is the model
class MLP(transformers.PreTrainedModel):

    config_class=MLPConfig

    # In the initialization method, one instantiates the layers
    # these will be, for the most part the trained parameters of the model
    def __init__(self,config):
        super().__init__(config)
        self.vocab_size=config.vocab_size #embedding matrix row count
        # Build and initialize embedding of vocab size +1 x hidden size (+1 because of the padding index 0!)
        self.embedding=torch.nn.Embedding(num_embeddings=self.vocab_size+1,embedding_dim=config.hidden_size,padding_idx=0)
        # Normally you would not initialize these yourself, but I have my reasons here ;)
        torch.nn.init.uniform_(self.embedding.weight.data,-0.001,0.001) #initialize the embeddings with small random values
        # Note! This is quite clever and keeps the embedding for 0, the padding, pure zeros
        # This takes care of the lower half of the network, now the upper half
        # Output layer: hidden size x output size
        self.output=torch.nn.Linear(in_features=config.hidden_size,out_features=config.nlabels)
        # Now we have the parameters of the model

    def forward(self,input_ids,labels=None): #nevermind the attention_mask, its time will come, data collator insists on adding it
        #sum up the embeddings of the items
        embedded=self.embedding(input_ids) #(batch,ids)->(batch,ids,embedding_dim)
        # Since the Embedding keeps the first row of the matrix pure zeros, we don't need to worry about the padding
        # so next we sum the embeddings across the word dimension
        # (batch,ids,embedding_dim) -> (batch,embedding_dim)
        embedded_summed=torch.sum(embedded,dim=1)
        logits=self.output(embedded_summed)

        # We have labels, so we ought to calculate the loss
        if labels is not None:
            loss=torch.nn.CrossEntropyLoss() #This loss is meant for classification, so let's use it
            # You run it as loss(model_output,correct_labels)
            return (loss(logits,labels),logits)
        else:
            # No labels, so just return the logits
            return (logits,)

In [13]:
# Configure the model:
mlp_config=MLPConfig(vocab_size=len(vectorizer.vocabulary_),hidden_size=1,nlabels=2)

In [14]:
#Make a model
mlp=MLP(mlp_config)
fake_batch=collator([dset_tokenized["train"][0],dset_tokenized["train"][1]])
mlp(**fake_batch) #** expands input_ids and labels as parameters of the call

(tensor(1.7214, grad_fn=<NllLossBackward0>),
 tensor([[-0.8101,  0.7184],
         [-0.8105,  0.7099]], grad_fn=<AddmmBackward0>))

In [15]:
# Set training arguments
# their names are mostly self-explanatory
trainer_args = transformers.TrainingArguments(
    "mlp_checkpoints", #save checkpoints here
    evaluation_strategy="steps",
    logging_strategy="steps",
    eval_steps=500,
    logging_steps=500,
    learning_rate=1e-4, #learning rate of the gradient descent
    max_steps=20000,
    load_best_model_at_end=True,
    per_device_train_batch_size=128
)

pprint(trainer_args)

TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_steps=500,
eval_strategy=steps,
evaluation_strategy=steps,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [16]:
# A metric for evaluating performance
import numpy as np
import evaluate

accuracy = evaluate.load("accuracy")

def compute_accuracy(outputs_and_labels):
    outputs, labels = outputs_and_labels
    predictions = np.argmax(outputs, axis=-1) #pick the index of the "winning" label
    return accuracy.compute(predictions=predictions, references=labels)

In [17]:
# Make a new model
mlp = MLP(mlp_config)


# Argument gives the number of steps of patience before early stopping
# i.e. training is stopped when the evaluation loss fails to improve
# certain number of times
early_stopping = transformers.EarlyStoppingCallback(5)

trainer = transformers.Trainer(
    model=mlp,
    args=trainer_args,
    train_dataset=dset_tokenized["train"],
    eval_dataset=dset_tokenized["test"].select(range(1000)), #make a smaller subset to evaluate on
    compute_metrics=compute_accuracy,
    data_collator=collator,
    callbacks=[early_stopping]
)

# FINALLY!
trainer.train()

max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss,Validation Loss,Accuracy
500,0.656600,0.617145,0.777000
1000,0.561300,0.533609,0.830000
1500,0.477300,0.470267,0.841000
2000,0.415000,0.424031,0.853000
2500,0.367400,0.390463,0.860000
3000,0.332100,0.366087,0.860000
3500,0.304900,0.347989,0.867000
4000,0.281800,0.333758,0.868000
4500,0.262700,0.323324,0.868000
5000,0.247500,0.314821,0.871000


TrainOutput(global_step=12000, training_loss=0.2688462518056234, metrics={'train_runtime': 278.0332, 'train_samples_per_second': 9207.535, 'train_steps_per_second': 71.934, 'total_flos': 15691920192.0, 'train_loss': 0.2688462518056234, 'epoch': 61.224489795918366})

In [18]:
#Evaluation
eval_results = trainer.evaluate(dset_tokenized["test"])

print(eval_results)

{'eval_loss': 0.28101739287376404, 'eval_accuracy': 0.88788, 'eval_runtime': 7.7577, 'eval_samples_per_second': 3222.621, 'eval_steps_per_second': 402.828, 'epoch': 61.224489795918366}


### 3.2 Hyperparameter optimization

### 3.3. Evaluation on test set

In [20]:
import optuna

def objective(trial):
    # Define the search space for hyperparameters
    learning_rate = trial.suggest_float("learning_rate", 1e-6, 1e-1, log=True)
    batch_size = trial.suggest_categorical("batch_size", [16, 64, 128, 256])

    trainer_args = transformers.TrainingArguments(
        "mlp_checkpoints", #save checkpoints here
        evaluation_strategy="steps",
        logging_strategy="steps",
        eval_steps=500,
        logging_steps=500,
        learning_rate=learning_rate, #learning rate of the gradient descent
        max_steps=10000, #I will keep this small, no time to wait too long :)
        load_best_model_at_end=True,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size
    )

    mlp = MLP(mlp_config)

    trainer = transformers.Trainer(
        model=mlp,
        args=trainer_args,
        train_dataset=dset_tokenized["train"],
        eval_dataset=dset_tokenized["test"].select(range(1000)), #make a smaller subset to evaluate on
        compute_metrics=compute_accuracy,
        data_collator=collator,
        callbacks=[early_stopping]
    )

    # Train the model and get the best validation loss
    trainer.train()
    eval_results = trainer.evaluate()
    return eval_results["eval_accuracy"] #let's try to maximize accuracy

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=7) #I will keep this small, no time to wait :)

[I 2024-06-16 11:03:52,221] A new study created in memory with name: no-name-740ac938-3c32-4320-9243-856de128ac00
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss,Validation Loss,Accuracy
500,0.131300,1.031386,0.827000
1000,0.024300,1.479061,0.838000
1500,0.014400,2.045264,0.824000
2000,0.003200,2.393821,0.824000
2500,0.000200,2.522709,0.826000
3000,0.000100,2.616001,0.827000


[I 2024-06-16 11:05:42,582] Trial 0 finished with value: 0.827 and parameters: {'learning_rate': 0.015139886222031091, 'batch_size': 256}. Best is trial 0 with value: 0.827.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss,Validation Loss,Accuracy
500,0.694700,0.695287,0.493000
1000,0.693700,0.694106,0.493000
1500,0.692700,0.692883,0.493000
2000,0.692200,0.691574,0.493000
2500,0.690100,0.690328,0.493000
3000,0.687900,0.689162,0.494000
3500,0.687100,0.688048,0.495000
4000,0.686200,0.686882,0.500000
4500,0.685100,0.685810,0.506000
5000,0.683800,0.684845,0.510000


[I 2024-06-16 11:06:40,216] Trial 1 finished with value: 0.587 and parameters: {'learning_rate': 1.0281648208483504e-05, 'batch_size': 16}. Best is trial 0 with value: 0.827.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


...

[I 2024-06-16 11:09:15,334] Trial 4 finished with value: 0.857 and parameters: {'learning_rate': 0.010158200712850077, 'batch_size': 128}. Best is trial 2 with value: 0.885.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss,Validation Loss,Accuracy
500,0.378600,0.312592,0.864000
1000,0.303400,0.321121,0.873000
1500,0.281700,0.329223,0.873000
2000,0.156300,0.389767,0.873000
2500,0.151200,0.413756,0.860000
3000,0.170700,0.388848,0.862000


[I 2024-06-16 11:09:33,668] Trial 5 finished with value: 0.864 and parameters: {'learning_rate': 0.01073032075563992, 'batch_size': 16}. Best is trial 2 with value: 0.885.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss,Validation Loss,Accuracy
500,0.225100,0.541437,0.841000
1000,0.075000,1.287505,0.843000
1500,0.032900,1.500652,0.825000
2000,0.024800,2.191846,0.826000
2500,0.016000,1.951905,0.830000
3000,0.011700,2.410964,0.819000


[I 2024-06-16 11:10:31,894] Trial 6 finished with value: 0.841 and parameters: {'learning_rate': 0.07297046313823954, 'batch_size': 128}. Best is trial 2 with value: 0.885.


In [21]:
# Print the best trial's hyperparameters and objective value
best_trial = study.best_trial
print(f"Best trial (number {best_trial.number}):")
print(f"  Value: {best_trial.value}")
print(f"  Params: {best_trial.params}")

# Print all trials' hyperparameters and objective values
print("\nAll trials:")
for trial in study.trials:
    print(f"  Trial {trial.number}:")
    print(f"    Value: {trial.value}")
    print(f"    Params: {trial.params}")

Best trial (number 2):
  Value: 0.885
  Params: {'learning_rate': 0.0025237117393771815, 'batch_size': 64}

All trials:
  Trial 0:
    Value: 0.827
    Params: {'learning_rate': 0.015139886222031091, 'batch_size': 256}
  Trial 1:
    Value: 0.587
    Params: {'learning_rate': 1.0281648208483504e-05, 'batch_size': 16}
  Trial 2:
    Value: 0.885
    Params: {'learning_rate': 0.0025237117393771815, 'batch_size': 64}
  Trial 3:
    Value: 0.855
    Params: {'learning_rate': 0.009208404757578328, 'batch_size': 128}
  Trial 4:
    Value: 0.857
    Params: {'learning_rate': 0.010158200712850077, 'batch_size': 128}
  Trial 5:
    Value: 0.864
    Params: {'learning_rate': 0.01073032075563992, 'batch_size': 16}
  Trial 6:
    Value: 0.841
    Params: {'learning_rate': 0.07297046313823954, 'batch_size': 128}


In [22]:
import optuna.visualization as vis
fig = vis.plot_parallel_coordinate(study)
fig.show()


In [23]:
fig = vis.plot_slice(study)
fig.show()

In [24]:
fig = vis.plot_contour(study, params=['learning_rate', 'batch_size'])
fig.show()

In [25]:
fig = vis.plot_optimization_history(study)
fig.show()

---

## 4. Results and summary

### 4.1 Corpus insights

(Briefly discuss what you learned about the corpus and its annotation)

### 4.2 Results

(Briefly summarize your results)

### 4.3 Relation to state of the art

(Compare your results to the state-of-the-art performance)

---

## 5. Bonus Task (optional)

### 5.1. Annotating out-of-domain documents

(Briefly describe the chosen out-of-domain documents)

(Briefly describe the process of annotation)

### 5.2 Conversion into dataset

In [ ]:
# Your code to convert the annotations into a dataset here

### 5.3. Model evaluation on out-of-domain test set

In [ ]:
# Your code to evaluate the model on the out-of-domain test set here

### 5.4 Bonus task results

(Present the results of the evaluation on the out-of-domain test set)

### 5.5. Annotated data

In [ ]:
# Include your annotated out-of-domain data here